In [51]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load only the first 1000 rows of the dataset
df = pd.read_csv('powerconsumption.csv', nrows=21000)
#df = pd.read_csv('powerconsumption.csv')

# Convert DateTime to datetime format
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Create a new 'date' column by extracting only the date part
df['date'] = df['Datetime'].dt.date

# Check the first few rows
df.head()

# Convert the Datetime column to datetime format
df['Datetime'] = pd.to_datetime(df['Datetime'])


# Create new columns
df['Year'] = df['Datetime'].dt.year
df['Month'] = df['Datetime'].dt.month
df['Day'] = df['Datetime'].dt.day
df['Hour'] = df['Datetime'].dt.hour

# Define time of day categories
def categorize_time_of_day(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Night'

df['TimeOfDay'] = df['Hour'].apply(categorize_time_of_day)

# Add day of the week (0 = Monday, 6 = Sunday)
df['Weekday'] = df['Datetime'].dt.weekday

# Add a column to indicate if it's a weekend
df['IsWeekend'] = df['Weekday'].apply(lambda x: 1 if x >= 5 else 0)

# Define the seasons
def categorize_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

df['Season'] = df['Month'].apply(categorize_season)

# Let's set the data types to numeric
df['Year'] = df['Year'].astype(int)
df['Weekday'] = df['Weekday'].astype(int)
df['IsWeekend'] = df['IsWeekend'].astype(int)

df.tail()

df.drop(columns=['Datetime'], inplace=True)
df.drop(columns=['date'], inplace=True)


def grab_col_names(dataframe, cat_th=10, car_th=20):
    # Categorical columns
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtype == "O"]
    
    # Numeric columns:
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    
    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    
    return cat_cols, num_cols

cat_cols, num_cols = grab_col_names(df)

cat_cols

df[cat_cols]

# Convert categorical variables to numerical values using One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=['TimeOfDay', 'Season'])

# Convert the columns resulting from One-Hot Encoding to integer values
df_encoded = df_encoded.astype(int)

# Display the results
df_encoded.head(6)


from sklearn.preprocessing import StandardScaler

# Separate features and target variables
features = df_encoded.drop(columns=['PowerConsumption_Zone1', 'PowerConsumption_Zone2', 'PowerConsumption_Zone3'])
targets = df_encoded[['PowerConsumption_Zone1', 'PowerConsumption_Zone2', 'PowerConsumption_Zone3']]

# Normalize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Combine scaled features and targets into a DataFrame
scaled_df = pd.DataFrame(scaled_features, columns=features.columns)
scaled_df[['PowerConsumption_Zone1', 'PowerConsumption_Zone2', 'PowerConsumption_Zone3']] = targets.values

import numpy as np

# Define sequence length (e.g., 24 hours)
sequence_length = 12

# Function to create sequences
def create_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data.iloc[i:i + sequence_length].drop(columns=['PowerConsumption_Zone1', 'PowerConsumption_Zone2', 'PowerConsumption_Zone3']).values)
        targets.append(data.iloc[i + sequence_length][['PowerConsumption_Zone1', 'PowerConsumption_Zone2', 'PowerConsumption_Zone3']].values)
    return np.array(sequences, dtype=np.float32), np.array(targets, dtype=np.float32)

# Create sequences
X, y = create_sequences(scaled_df, sequence_length)

# Split into training, validation, and test sets
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)
test_size = len(X) - train_size - val_size

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size + val_size], y[train_size:train_size + val_size]
X_test, y_test = X[train_size + val_size:], y[train_size + val_size:]

scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val = scaler_X.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train)
y_val = scaler_y.transform(y_val)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Test data shape: {X_test.shape}")

Observations: 21000
Variables: 16
cat_cols: 2
num_cols: 14
Training data shape: (14691, 12, 17)
Validation data shape: (3148, 12, 17)
Test data shape: (3149, 12, 17)


In [52]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the Vanilla Transformer model
class VanillaTransformer(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim):
        super(VanillaTransformer, self).__init__()
        self.linear_projection = nn.Linear(input_dim, model_dim)  # Project input to model_dim
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(model_dim, output_dim)

    def forward(self, src):
        src = src.permute(1, 0, 2)  # Transformer expects (seq_len, batch_size, input_dim)
        src = self.linear_projection(src)  # Project input to model_dim
        output = self.transformer_encoder(src)
        output = self.fc(output[-1, :, :])  # Use the last time step's output
        return output


In [53]:
# Hyperparameters
input_dim = X_train.shape[2]  # Number of features (19)
model_dim = 64  # Desired embedding dimension
num_heads = 4  # Number of attention heads
num_layers = 2  # Number of transformer layers
output_dim = 3  # Number of target variables (Zone1, Zone2, Zone3)

# Initialize the model
model = VanillaTransformer(input_dim, model_dim, num_heads, num_layers, output_dim)

/Users/ibti/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [54]:
# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [55]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    print(f"Starting Epoch {epoch+1}")
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    print("Forward pass completed")
    loss = criterion(outputs, y_train_tensor)
    print("Loss computed")
    loss.backward()
    print("Backward pass completed")
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Starting Epoch 1
Forward pass completed
Loss computed
Backward pass completed
Epoch 1, Loss: 1.3925999402999878
Starting Epoch 2
Forward pass completed
Loss computed
Backward pass completed
Epoch 2, Loss: 1.025205373764038
Starting Epoch 3
Forward pass completed
Loss computed
Backward pass completed
Epoch 3, Loss: 0.5713690519332886
Starting Epoch 4
Forward pass completed
Loss computed
Backward pass completed
Epoch 4, Loss: 0.46226766705513
Starting Epoch 5
Forward pass completed
Loss computed
Backward pass completed
Epoch 5, Loss: 0.4880897104740143
Starting Epoch 6
Forward pass completed
Loss computed
Backward pass completed
Epoch 6, Loss: 0.42542964220046997
Starting Epoch 7
Forward pass completed
Loss computed
Backward pass completed
Epoch 7, Loss: 0.29303595423698425
Starting Epoch 8
Forward pass completed
Loss computed
Backward pass completed
Epoch 8, Loss: 0.2421240359544754
Starting Epoch 9
Forward pass completed
Loss computed
Backward pass completed
Epoch 9, Loss: 0.2886546552